In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# import my venv

In [2]:
!source /content/drive/MyDrive/colab_env/bin/activate

In [3]:
! pip install -q pyet swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 20.4 MB/s eta 0:00:00


In [4]:
import pandas as pd
import pyarrow
import pyet
import math
import datetime
import swifter
import numpy as np
import glob
import os
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
from scipy import stats
import calendar

# calculate ETo

## import prism data for each of ameri flux stations

In [5]:
prism_data_daily = pd.read_parquet('/content/drive/MyDrive/EC_Tower/merged_prism_data_par_for_ameri_stations.parquet', engine='pyarrow')


## convert lat to radian and set time as TIMESTAMP

In [6]:
# convert lat to radian
prism_data_daily["latitude_in_radian"] = list(map(math.radians, prism_data_daily["latitude"]))

prism_data_daily['time'] = pd.to_datetime(prism_data_daily['date'],format= "%Y-%m-%d" )

prism_data_daily.set_index('time', inplace=True)
prism_data_daily

,system:index,date,point_id,ppt,.geo,tdmean,tmean,vpdmax,vpdmin,tmin,tmax,latitude,longitude,Site_ID,General_classification,Elevation,Energy_balance,Land_cover_details,Land_cover_type,latitude_in_radian
time,,,,,,,,,,,,,,,,,,,,
2008-12-31,0_20090101,2008-12-31,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.717,5.503000,0.912,0.239,4.565,6.442,36.4587,-119.5801,MB_Pch,Croplands,90.0,1.00,Peach,Orchards,0.636324
2009-01-01,0_20090102,2009-01-01,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.025,5.944000,1.700,0.494,4.836,7.053,36.4587,-119.5801,MB_Pch,Croplands,90.0,1.00,Peach,Orchards,0.636324
2009-01-02,0_20090103,2009-01-02,1,2.898,"{""type"":""MultiPoint"",""coordinates"":[]}",4.454,6.111000,2.067,0.370,4.656,7.567,36.4587,-119.5801,MB_Pch,Croplands,90.0,1.00,Peach,Orchards,0.636324
2009-01-03,0_20090104,2009-01-03,1,0.228,"{""type"":""MultiPoint"",""coordinates"":[]}",3.300,5.052000,3.851,0.202,0.441,9.665,36.4587,-119.5801,MB_Pch,Croplands,90.0,1.00,Peach,Orchards,0.636324
2009-01-04,0_20090105,2009-01-04,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",1.732,4.622000,3.458,0.290,0.316,8.929,36.4587,-119.5801,MB_Pch,Croplands,90.0,1.00,Peach,Orchards,0.636324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-26,12_20201227,2020-12-26,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",6.645,11.471399,8.840,0.435,6.817,16.126,38.2006,-122.0264,US-Srr,Wetland/Riparian,8.0,0.86,Brackish tidal marsh,Wetlands,0.666726
2020-12-27,12_20201228,2020-12-27,13,1.227,"{""type"":""MultiPoint"",""coordinates"":[]}",4.315,9.327499,7.957,0.464,4.567,14.088,38.2006,-122.0264,US-Srr,Wetland/Riparian,8.0,0.86,Brackish tidal marsh,Wetlands,0.666726
2020-12-28,12_20201229,2020-12-28,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.685,9.029900,6.475,0.323,4.429,13.631,38.2006,-122.0264,US-Srr,Wetland/Riparian,8.0,0.86,Brackish tidal marsh,Wetlands,0.666726


## functions for calculation rs

In [7]:
# Solar Constant [ MJ m-2 min-1]
SOLAR_CONSTANT = 0.0820

def standard_date_to_Julian_day (
    standard_date_in_gregorian : str
    ) -> int:

    """
    Description
    -----------
    calculate Julian Day with standard date
    Ref:https://rafatieppo.github.io/post/2018_12_01_juliandate/
    ----------

    standard_date_in_gregorian : str
        Date with the specified standard

    Returns
    -------
    Julian Day : int
        Number of days of the year taking into account the leap year
    """

    fmt='%Y-%m-%d'
    sdtdate = datetime.datetime.strptime(standard_date_in_gregorian, fmt)
    sdtdate = sdtdate.timetuple()
    jdate = sdtdate.tm_yday

    return(jdate)


def inverse_relative_distance_earth_sun(
    julian_date: int
) -> float:

    """
    Description
    -----------
    calculate Inverse Relative Distance Earth Sun With Julian Date - eq 23 FAO56
    ----------


    Julian Day : int
        Number of days of the year taking into account the leap year

    Returns
    -------
    inverse_relative_distance_earth_sun : float
        inverse relative distance earth sun in Radian
    """

    return 1 + (0.033 * np.cos((2 * np.pi * julian_date)/365))

def solar_declination(
    julian_date: int
) -> float:

    """
    Description
    -----------
    calculate Solar Declination With Julian Date - eq 24 FAO56
    ----------

    julian_date : int
        Number of days of the year taking into account the leap year

    Returns
    -------
    solar_declination : float
        solar_declination in Radian
    """

    return 0.409 * np.sin(((2 * np.pi * julian_date) / 365) - 1.39)

def sunset_hour_angle(
    latitude : float,
    solar_declination : float
) -> float:

    """
    Description
    -----------
    calculate Sunset Hour Angle With Latitude and Solar Declination - eq 25 FAO56
    ----------

    latitude: float
        latitude in Radian
    solar_declination : float
        solar_declination in Radian

    Returns
    -------
    sunset_hour_angle : float
        sunset hour angle in Radian
    """

    return np.arccos(-np.tan(latitude) * np.tan(solar_declination))

def extraterrestrial_radiation(
    inverse_relative_distance_earth_sun : float,
    sunset_hour_angle : float,
    latitude : float,
    solar_declination : float
) -> float:

    """
    Description
    -----------
    calculate Extraterrestrial Radiation - eq 28 FAO56
    ----------

    inverse_relative_distance_earth_sun : float
        inverse relative distance earth sun in Radian
    sunset_hour_angle : float
        sunset hour angle in Radian
    latitude: float
        latitude in Radian
    solar_declination : float
        solar_declination in Radian

    Returns
    -------
    extraterrestrial_radiation : float
            extraterrestrial radiation in MJ/m**2/day
    """

    temp_1 = ((24 * 60 )/ np.pi) * SOLAR_CONSTANT * inverse_relative_distance_earth_sun

    temp_2 = sunset_hour_angle * np.sin(latitude) * np.sin(solar_declination)

    temp_3 = np.cos(latitude)* np.cos(solar_declination) * np.sin(sunset_hour_angle)

    return temp_1 * (temp_2 + temp_3)

def solar_or_Short_wave_Radiation_Hargreaves(
    extraterrestrial_radiation : float,
    tmax : float,
    tmin : float,
    Adjustment_coefficient_or_K_RS : float
)-> float:

    """
    Description
    -----------
    calculate Solar or shortwave radiation with max and min Tempreture - eq 50 FAO56
    ----------

    extraterrestrial_radiation : float
        extraterrestrial radiation in MJ/m**2/day
    tmax : float
        Maximum Temperature in celsius
    tmin : float
        Minimum Temperature in celsius
    Adjustment_coefficient_or_K_RS : float
        Adjustment coefficient in C**-0.5 -- between 0.16 to 0.19
            for interior locations, where land mass dominates and air masses are not strongly
            influenced by a large water body, kRs ≅ 0.16;
            for coastal locations, situated on or adjacent to the coast of a large land mass and where
            air masses are influenced by a nearby water body, kRs ≅ 0.19

    Returns
    -------
    Solar or shortwave radiation : float
        Solar or shortwave radiation in MJ/m**2/day
    """
    temp_1 = extraterrestrial_radiation


    return Adjustment_coefficient_or_K_RS * np.sqrt(tmax - tmin) * temp_1

## calculate some asset parameter

In [8]:
Adjustment_coefficient_or_K_RS = 0.175


prism_data_daily['julian_day'] = prism_data_daily.apply(lambda row: standard_date_to_Julian_day(row['date']), axis=1)
prism_data_daily['dr'] = prism_data_daily.apply(lambda row: inverse_relative_distance_earth_sun(row['julian_day']), axis=1)
prism_data_daily['delta'] = prism_data_daily.apply(lambda row: solar_declination(row['julian_day']), axis=1)
prism_data_daily['omega_s'] = prism_data_daily.apply(lambda row: sunset_hour_angle(row['latitude_in_radian'], row["delta"]), axis=1)
prism_data_daily['ra'] = prism_data_daily.apply(lambda row: extraterrestrial_radiation(row['dr'], row["omega_s"], row["latitude_in_radian"], row["delta"]), axis=1)
prism_data_daily['rs'] = prism_data_daily.apply(lambda row: solar_or_Short_wave_Radiation_Hargreaves(row['ra'], row["tmax"], row["tmin"], Adjustment_coefficient_or_K_RS), axis=1)


prism_data_daily

,system:index,date,point_id,ppt,.geo,tdmean,tmean,vpdmax,vpdmin,tmin,...,Energy_balance,Land_cover_details,Land_cover_type,latitude_in_radian,julian_day,dr,delta,omega_s,ra,rs
time,,,,,,,,,,,,,,,,,,,,,
2008-12-31,0_20090101,2008-12-31,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.717,5.503000,0.912,0.239,4.565,...,1.00,Peach,Orchards,0.636324,366,1.032995,-0.401008,1.252175,16.025685,3.842260
2009-01-01,0_20090102,2009-01-01,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.025,5.944000,1.700,0.494,4.836,...,1.00,Peach,Orchards,0.636324,1,1.032995,-0.401008,1.252175,16.025685,4.175779
2009-01-02,0_20090103,2009-01-02,1,2.898,"{""type"":""MultiPoint"",""coordinates"":[]}",4.454,6.111000,2.067,0.370,4.656,...,1.00,Peach,Orchards,0.636324,2,1.032980,-0.399564,1.253500,16.080602,4.801329
2009-01-03,0_20090104,2009-01-03,1,0.228,"{""type"":""MultiPoint"",""coordinates"":[]}",3.300,5.052000,3.851,0.202,0.441,...,1.00,Peach,Orchards,0.636324,3,1.032956,-0.398001,1.254930,16.139901,8.578247
2009-01-04,0_20090105,2009-01-04,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",1.732,4.622000,3.458,0.290,0.316,...,1.00,Peach,Orchards,0.636324,4,1.032922,-0.396320,1.256466,16.203566,8.321965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-26,12_20201227,2020-12-26,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",6.645,11.471399,8.840,0.435,6.817,...,0.86,Brackish tidal marsh,Wetlands,0.666726,361,1.032922,-0.406440,1.225258,14.737636,7.868961
2020-12-27,12_20201228,2020-12-27,13,1.227,"{""type"":""MultiPoint"",""coordinates"":[]}",4.315,9.327499,7.957,0.464,4.567,...,0.86,Brackish tidal marsh,Wetlands,0.666726,362,1.032956,-0.405594,1.226097,14.770615,7.975867
2020-12-28,12_20201229,2020-12-28,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.685,9.029900,6.475,0.323,4.429,...,0.86,Brackish tidal marsh,Wetlands,0.666726,363,1.032980,-0.404627,1.227054,14.808078,7.861001


## calculate ETo based 4 methods

In [9]:
# prism_data_daily["blaney_criddle"] = pyet.blaney_criddle(
#     tmean = prism_data_daily["tmean"],
#     lat = prism_data_daily["latitude_in_radian"]
# )

# prism_data_daily["hamon"] = pyet.hamon(
#     tmean = prism_data_daily["tmean"],
#     lat = prism_data_daily["latitude_in_radian"]
# )

prism_data_daily["oudin"] = pyet.oudin(
    tmean = prism_data_daily["tmean"],
    lat = prism_data_daily["latitude_in_radian"]
)

prism_data_daily["hargreaves"] = pyet.hargreaves(
    tmean = prism_data_daily["tmean"],
    tmax = prism_data_daily["tmax"],
    tmin = prism_data_daily["tmin"],
    lat = prism_data_daily["latitude_in_radian"]
)

prism_data_daily["abtew"] = pyet.abtew(
    tmean = prism_data_daily["tmean"],
    rs = prism_data_daily["rs"]
)


prism_data_daily["mcguinness_bordne"] = pyet.mcguinness_bordne(
    tmean = prism_data_daily["tmean"],
    lat = prism_data_daily["latitude_in_radian"]
)

# prism_data_daily["jensen_haise"] = pyet.jensen_haise(
#     tmean = prism_data_daily["tmean"],
#     lat = prism_data_daily["latitude_in_radian"]
# )
prism_data_daily = prism_data_daily.reset_index()

# read informations each of ameri flux stations and cocantenate them

In [10]:
# Specify the directory path and file pattern

file_path = '/content/drive/MyDrive/EC_Tower/stations/*.csv'
csv_files = glob.glob(file_path)

dfs = []

for file in csv_files:

    ts_data = pd.read_csv(file)

    # Get the file name without extension
    file_name = os.path.splitext(os.path.basename(file))[0]
    # Remove '_daily_data' suffix
    station_name = file_name.replace('_daily_data', '')

    # ts_data["Site_ID"] = file.removesuffix("_daily_data.csv")[41:]
    ts_data["Site_ID"] = station_name

    dfs.append(ts_data)

merged_df = pd.concat(dfs, ignore_index=True)
merged_df['time'] = pd.to_datetime(merged_df['date'],format= "%Y-%m-%d" )
merged_df



,date,rh,H,sw_in,t_dew,vp,theta_1,lw_out,LE,wd,...,ET_fill_val,Site_ID,H_qc_flag,Rn_qc_flag,LE_qc_flag,t_avg_qc_flag,vpd_qc_flag,G_qc_flag,ppt,time
0,2012-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,US-Tw2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-01-01
1,2012-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,US-Tw2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-01-02
2,2012-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,US-Tw2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-01-03
3,2012-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,US-Tw2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-01-04
4,2012-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,US-Tw2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20138,2018-05-31,58.591875,74.299147,346.442861,11.252931,1.361338,26.386760,404.013732,60.627478,242.604626,...,NaN,US-Tw3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2018-05-31
20139,2018-06-01,54.740417,57.072189,354.218242,12.342688,1.462196,26.324656,420.252554,75.335373,282.614125,...,NaN,US-Tw3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2018-06-01
20140,2018-06-02,45.873125,34.105066,356.110903,12.915099,1.533126,26.316750,438.610560,109.118007,238.779884,...,NaN,US-Tw3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2018-06-02
20141,2018-06-03,47.508958,15.475881,353.838460,13.621733,1.604567,26.276646,443.144378,122.170050,225.033641,...,NaN,US-Tw3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2018-06-03


# add observed ET real in ameri flux stations to prism data set

In [11]:

# Merge the DataFrames
result = pd.merge(prism_data_daily, merged_df[['date', 'Site_ID', 'ET_fill']],
                  on=['date', 'Site_ID'],
                  how='left')

result

,time,system:index,date,point_id,ppt,.geo,tdmean,tmean,vpdmax,vpdmin,...,dr,delta,omega_s,ra,rs,oudin,hargreaves,abtew,mcguinness_bordne,ET_fill
0,2008-12-31,0_20090101,2008-12-31,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.717,5.503000,0.912,0.239,...,1.032995,-0.401008,1.252175,16.025685,3.842260,0.676516,0.472973,0.818486,0.994479,NaN
1,2009-01-01,0_20090102,2009-01-01,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.025,5.944000,1.700,0.494,...,1.032995,-0.401008,1.252175,16.025685,4.175779,0.705217,0.523976,0.889905,1.036669,NaN
2,2009-01-02,0_20090103,2009-01-02,1,2.898,"{""type"":""MultiPoint"",""coordinates"":[]}",4.454,6.111000,2.067,0.370,...,1.032980,-0.399564,1.253500,16.080602,4.801329,0.718546,0.606803,1.023379,1.056262,NaN
3,2009-01-03,0_20090104,2009-01-03,1,0.228,"{""type"":""MultiPoint"",""coordinates"":[]}",3.300,5.052000,3.851,0.202,...,1.032956,-0.398001,1.254930,16.139901,8.578247,0.651802,1.035083,1.826573,0.958149,NaN
4,2009-01-04,0_20090105,2009-01-04,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",1.732,4.622000,3.458,0.290,...,1.032922,-0.396320,1.256466,16.203566,8.321965,0.626125,0.984862,1.771280,0.920404,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56974,2020-12-26,12_20201227,2020-12-26,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",6.645,11.471399,8.840,0.435,...,1.032922,-0.406440,1.225258,14.737636,7.868961,0.981236,1.223674,1.685809,1.442417,NaN
56975,2020-12-27,12_20201228,2020-12-27,13,1.227,"{""type"":""MultiPoint"",""coordinates"":[]}",4.315,9.327499,7.957,0.464,...,1.032956,-0.405594,1.226097,14.770615,7.975867,0.853682,1.147109,1.705223,1.254913,NaN
56976,2020-12-28,12_20201229,2020-12-28,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.685,9.029900,6.475,0.323,...,1.032980,-0.404627,1.227054,14.808078,7.861001,0.837833,1.117869,1.680189,1.231615,NaN
56977,2020-12-29,12_20201230,2020-12-29,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",-1.101,9.277900,15.402,0.592,...,1.032995,-0.403540,1.228128,14.850017,10.391113,0.855260,1.491673,2.221492,1.257232,NaN


# functions for budyko curve and calculate ET real based on precipitation and ETo

In [12]:
def budyko(pet, p):
    # from Budyko 1958
    # p,pet = X
    if p == 0:
        x = 0
    else:
        # x = (1 + (0.5 * (pet / p)** (-1.5)))**(-1)
        x = math.sqrt(((pet/p)*math.tanh(p/pet))*(1-math.exp(-pet/p)))
    return x


def aet(ratio, p):
  return ratio * p

## calculate ET real based budyko curve in 4 methods

In [13]:


result['ratio_budyko_oudin'] = result.apply(lambda row: budyko(row['oudin'], row["ppt"]), axis=1)

result['aet_budyko_oudin'] = result.apply(lambda row: aet(row['ratio_budyko_oudin'], row["ppt"]), axis=1)


result['ratio_budyko_hargreaves'] = result.apply(lambda row: budyko(row['hargreaves'], row["ppt"]), axis=1)

result['aet_budyko_hargreaves'] = result.apply(lambda row: aet(row['ratio_budyko_hargreaves'], row["ppt"]), axis=1)


result['ratio_budyko_abtew'] = result.apply(lambda row: budyko(row['abtew'], row["ppt"]), axis=1)

result['aet_budyko_abtew'] = result.apply(lambda row: aet(row['ratio_budyko_abtew'], row["ppt"]), axis=1)


result['ratio_budyko_mcguinness_bordne'] = result.apply(lambda row: budyko(row['mcguinness_bordne'], row["ppt"]), axis=1)

result['aet_budyko_mcguinness_bordne'] = result.apply(lambda row: aet(row['ratio_budyko_mcguinness_bordne'], row["ppt"]), axis=1)


result




,time,system:index,date,point_id,ppt,.geo,tdmean,tmean,vpdmax,vpdmin,...,mcguinness_bordne,ET_fill,ratio_budyko_oudin,aet_budyko_oudin,ratio_budyko_hargreaves,aet_budyko_hargreaves,ratio_budyko_abtew,aet_budyko_abtew,ratio_budyko_mcguinness_bordne,aet_budyko_mcguinness_bordne
0,2008-12-31,0_20090101,2008-12-31,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.717,5.503000,0.912,0.239,...,0.994479,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2009-01-01,0_20090102,2009-01-01,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.025,5.944000,1.700,0.494,...,1.036669,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2009-01-02,0_20090103,2009-01-02,1,2.898,"{""type"":""MultiPoint"",""coordinates"":[]}",4.454,6.111000,2.067,0.370,...,1.056262,NaN,0.233268,0.676011,0.198876,0.576341,0.323011,0.936085,0.332279,0.962946
3,2009-01-03,0_20090104,2009-01-03,1,0.228,"{""type"":""MultiPoint"",""coordinates"":[]}",3.300,5.052000,3.851,0.202,...,0.958149,NaN,0.951844,0.217020,0.986726,0.224974,0.997250,0.227373,0.983290,0.224190
4,2009-01-04,0_20090105,2009-01-04,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",1.732,4.622000,3.458,0.290,...,0.920404,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56974,2020-12-26,12_20201227,2020-12-26,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",6.645,11.471399,8.840,0.435,...,1.442417,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
56975,2020-12-27,12_20201228,2020-12-27,13,1.227,"{""type"":""MultiPoint"",""coordinates"":[]}",4.315,9.327499,7.957,0.464,...,1.254913,NaN,0.558132,0.684828,0.669477,0.821448,0.802162,0.984252,0.701850,0.861170
56976,2020-12-28,12_20201229,2020-12-28,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.685,9.029900,6.475,0.323,...,1.231615,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
56977,2020-12-29,12_20201230,2020-12-29,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",-1.101,9.277900,15.402,0.592,...,1.257232,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# functions for calculate metrics inedex

In [14]:
# Assuming your DataFrame is named 'df' and the two columns are 'actual' and 'predicted'
def calculate_metrics(df, actual_col, predicted_col):
    df = df.dropna()
    actual = df[actual_col]
    predicted = df[predicted_col]

    rmse = np.sqrt(mean_squared_error(actual, predicted))

    nse = 1 - (np.sum((actual - predicted) ** 2) / np.sum((actual - np.mean(actual)) ** 2))

    nrmse = rmse / (np.max(actual) - np.min(actual))

    r = stats.pearsonr(predicted,actual)[0]
    r2= r**2

    return rmse, nse, nrmse, r2

## import informations of ameri flux stations like length time series each station

In [15]:
len_date_of_data = pd.read_csv('/content/drive/MyDrive/EC_Tower/station_metadata_len.csv')

### convert last day of tie series as TIMESTAMP

In [16]:
def get_month_range(year, month):


    # Get the last day of the month
    _, last_day_of_month = calendar.monthrange(year, month)
    last_day = datetime.date(year, month, last_day_of_month)

    return last_day

### calculate nan value in each of stations

In [17]:
count_of_nan_for_stations = {}

list_of_stations = result.drop_duplicates(subset='Site_ID', keep='first')

for st in list_of_stations["Site_ID"].tolist():

  m_s = len_date_of_data.loc[len_date_of_data["Site ID"]==str(st)]["Period of record"].values[0][:2]

  y_s = len_date_of_data.loc[len_date_of_data["Site ID"]==str(st)]["Period of record"].values[0][3:7]
  # [8:]

  m_e = len_date_of_data.loc[len_date_of_data["Site ID"]==str(st)]["Period of record"].values[0][8:10]

  y_e = len_date_of_data.loc[len_date_of_data["Site ID"]==str(st)]["Period of record"].values[0][11:]

  first_day = datetime.date(int(y_s), int(m_s), 1)

  last_date = get_month_range(int(y_e), int(m_e))

  calnan = result.loc[result['date'].between(str(first_day), str(last_date))]
  calnan = calnan.loc[calnan['Site_ID']==str(st)]

  nan_count = calnan['ET_fill'].isna().sum()



  count_of_nan_for_stations[f'{str(st)}'] = nan_count




## filter data just for is value

In [18]:
filtered_df = result[result['ET_fill'].apply(lambda x: isinstance(x, float) and not np.isnan(x))]

filtered_df = filtered_df[filtered_df['ratio_budyko_hargreaves'].apply(lambda x: isinstance(x, float) and not np.isnan(x))]

filtered_df = filtered_df[filtered_df['ratio_budyko_hargreaves'].apply(lambda x: isinstance(x, float) and x != 0)]

## extract observed and simulation columns

In [19]:
observed_col = 'ET_fill'
method_cols = [col for col in filtered_df.columns if col.startswith('aet_')]


## calculate metrics index for daily data each stations

In [20]:
# Function to calculate metrics for a specific time scale
def calculate_timescale_metrics(data, timescale):
    results = []
    for method in method_cols:
        metrics = data.groupby('Site_ID').apply(lambda x: calculate_metrics(x, observed_col, method))
        metrics = metrics.reset_index()
        metrics['Method'] = method
        metrics['Timescale'] = timescale
        results.append(metrics)
    return pd.concat(results, ignore_index=True)



# Daily calculations
daily_metrics = calculate_timescale_metrics(filtered_df, 'Daily')

daily_metrics["number_of_days_nan"] = daily_metrics['Site_ID'].map(count_of_nan_for_stations)

daily_metrics = pd.merge(daily_metrics, filtered_df[['Site_ID','latitude', 'General_classification', 'Elevation', 'Land_cover_details', 'Land_cover_type']],
                  on=['Site_ID'],
                  how='left')
daily_data = filtered_df[['time','latitude','Site_ID', 'General_classification', 'Elevation','Land_cover_details', 'Land_cover_type','aet_budyko_oudin',
             'aet_budyko_hargreaves', 'aet_budyko_abtew', 'aet_budyko_mcguinness_bordne', 'ET_fill']]

In [21]:
daily_data_1 = pd.merge(daily_data, merged_df[['Site_ID', 'time','ppt', 'ASCE_ETo']],
                  on=['Site_ID', 'time'],
                  how='left')


daily_data_1 = daily_data_1.rename(columns={'ppt': 'ppt_ameri'})

daily_data_1 = pd.merge(daily_data_1, prism_data_daily[['Site_ID', 'time','ppt', 'oudin', 'hargreaves', 'abtew', 'mcguinness_bordne']],
                  on=['Site_ID', 'time'],
                  how='left')

daily_data_1

,time,latitude,Site_ID,General_classification,Elevation,Land_cover_details,Land_cover_type,aet_budyko_oudin,aet_budyko_hargreaves,aet_budyko_abtew,aet_budyko_mcguinness_bordne,ET_fill,ppt_ameri,ASCE_ETo,ppt,oudin,hargreaves,abtew,mcguinness_bordne
0,2012-04-10,36.4587,MB_Pch,Croplands,90.0,Peach,Orchards,2.462327,3.408489,3.892481,3.309454,3.601669,NaN,NaN,6.001,2.791157,4.284434,5.302270,4.103000
1,2012-04-11,36.4587,MB_Pch,Croplands,90.0,Peach,Orchards,2.147459,2.475161,3.353678,3.067134,2.424285,NaN,NaN,9.757,2.274162,2.646425,3.694394,3.343019
2,2012-04-12,36.4587,MB_Pch,Croplands,90.0,Peach,Orchards,2.216234,2.642178,3.592554,3.193575,2.327114,NaN,NaN,13.397,2.312553,2.780581,3.857771,3.399453
3,2012-04-13,36.4587,MB_Pch,Croplands,90.0,Peach,Orchards,1.991747,1.826559,2.671896,2.886146,2.049389,NaN,NaN,16.407,2.054412,1.879101,2.786116,3.019986
4,2012-04-25,36.4587,MB_Pch,Croplands,90.0,Peach,Orchards,2.728037,2.923769,2.994257,3.231196,2.962212,NaN,NaN,3.982,3.886146,4.471146,4.714195,5.712635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3062,2017-06-07,38.2006,US-Srr,Wetland/Riparian,8.0,Brackish tidal marsh,Wetlands,1.316575,1.342311,1.351500,1.358185,5.428622,NaN,4.910524,1.380,4.019611,4.890679,5.395064,5.908828
3063,2017-06-08,38.2006,US-Srr,Wetland/Riparian,8.0,Brackish tidal marsh,Wetlands,0.003000,0.003000,0.003000,0.003000,1.961380,NaN,3.159529,0.003,3.805932,3.505136,4.004205,5.594720
3064,2017-06-10,38.2006,US-Srr,Wetland/Riparian,8.0,Brackish tidal marsh,Wetlands,2.045932,2.210257,2.289231,2.274335,5.301356,NaN,5.684899,2.504,3.663386,4.750406,5.562129,5.385177
3065,2017-06-11,38.2006,US-Srr,Wetland/Riparian,8.0,Brackish tidal marsh,Wetlands,0.577167,0.579356,0.579981,0.579586,4.792554,NaN,4.662564,0.581,3.337412,4.598271,5.698883,4.905995


## calculate metrics index for decade data each stations

In [22]:
def cal_decade_month(day):

  if day <= 10:
    de = 1
  elif day > 20:
    de = 3
  else:
    de = 2
  return de



result["Month"] = result["time"].apply(lambda x: x.month)
result["Year"] = result["time"].apply(lambda x: x.year)
result["decade_of_this_month"] = result["time"].apply(lambda x: cal_decade_month(x.day))
# filtered_df["decade_of_this_month"] = filtered_df["time"].apply(lambda x: cal_decade_month(x.day))

result_for_decade = result.groupby(["Site_ID", "Year", "Month", "decade_of_this_month"]).agg(
  number_row = ("ET_fill", "size"),
  number_value =  ("ET_fill", "count"),
)
result_for_decade.reset_index(inplace=True)

result_for_decade["percent_of_value"] = result_for_decade["number_value"] / result_for_decade["number_row"]


daily_data_1["Month"] = daily_data_1["time"].apply(lambda x: x.month)
daily_data_1["Year"] = daily_data_1["time"].apply(lambda x: x.year)
daily_data_1["decade_of_this_month"] = daily_data_1["time"].apply(lambda x: cal_decade_month(x.day))



decade_data = daily_data_1.groupby(['Site_ID', 'Year', 'Month','decade_of_this_month']).agg({'aet_budyko_oudin': 'sum',
                                                                                             'aet_budyko_hargreaves': 'sum',
                                                                                             'aet_budyko_abtew': 'sum',
                                                                                             'aet_budyko_mcguinness_bordne': 'sum',
                                                                                             'ET_fill': 'sum',
                                                                                             'ppt_ameri' : 'sum',
                                                                                             'ASCE_ETo' : 'sum',
                                                                                             'ppt' : 'sum',
                                                                                             'oudin' : 'sum',
                                                                                             'hargreaves' : 'sum',
                                                                                             'abtew' : 'sum',
                                                                                             'mcguinness_bordne' : 'sum'}).reset_index()

stations_unique_info = daily_data_1.drop_duplicates(subset=['Site_ID'])

stations_unique_info = stations_unique_info.drop(['time',
                                                  'aet_budyko_oudin',
                                                  'aet_budyko_hargreaves',
                                                  'aet_budyko_abtew',
                                                  'aet_budyko_mcguinness_bordne',
                                                  'ET_fill',
                                                  'ppt_ameri',
                                                  'ASCE_ETo',
                                                  'ppt',
                                                  'oudin',
                                                  'hargreaves',
                                                  'abtew',
                                                  'mcguinness_bordne',
                                                  'Month',
                                                  'Year',
                                                  'decade_of_this_month'], axis =1)



decade_data = decade_data.merge(
    stations_unique_info,
    left_on=['Site_ID'],
    right_on=['Site_ID'],
)


decade_metrics = calculate_timescale_metrics(decade_data, 'decade')


decade_metrics = decade_metrics.merge(
    stations_unique_info,
    left_on=['Site_ID'],
    right_on=['Site_ID'],
)




# decade_metrics = pd.merge(decade_metrics, result_for_decade[['Site_ID', 'Year', 'Month', 'decade_of_this_month','number_row', 'number_value', 'percent_of_value']],
#                   on=['Site_ID', 'Year', 'Month', 'decade_of_this_month'],
#                   how='left')
decade_data



,Site_ID,Year,Month,decade_of_this_month,aet_budyko_oudin,aet_budyko_hargreaves,aet_budyko_abtew,aet_budyko_mcguinness_bordne,ET_fill,ppt_ameri,...,ppt,oudin,hargreaves,abtew,mcguinness_bordne,latitude,General_classification,Elevation,Land_cover_details,Land_cover_type
0,Almond_High,2016,10,2,0.859298,0.864168,0.872615,0.893563,5.587778,0.000,...,0.913,2.402789,2.500525,2.700058,3.532099,36.1697,Croplands,147.0,Almond,Orchards
1,Almond_High,2016,10,3,4.713892,5.530204,5.944873,6.475463,8.936655,37.592,...,21.902,5.962023,7.333244,8.291621,8.764173,36.1697,Croplands,147.0,Almond,Orchards
2,Almond_High,2016,11,2,2.324289,2.793675,3.312961,3.098271,4.485196,33.020,...,18.688,2.824190,3.735073,4.644811,4.151559,36.1697,Croplands,147.0,Almond,Orchards
3,Almond_High,2016,11,3,1.133692,1.421623,1.814044,1.482463,3.677810,1.524,...,2.908,2.073955,2.700008,3.906876,3.048714,36.1697,Croplands,147.0,Almond,Orchards
4,Almond_High,2016,12,1,1.217314,1.460342,1.744453,1.458898,1.706984,0.000,...,2.075,2.059063,2.522458,3.590562,3.026823,36.1697,Croplands,147.0,Almond,Orchards
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
920,US-Twt,2017,2,2,8.007416,7.127096,9.059095,10.999122,4.693896,42.300,...,57.379,9.214771,8.415534,11.252221,13.545713,38.1087,Croplands,-7.0,Rice,Annual crops
921,US-Twt,2017,2,3,1.560582,1.851748,2.429529,2.122794,3.782659,4.500,...,5.449,2.738527,3.791166,5.613883,4.025635,38.1087,Croplands,-7.0,Rice,Annual crops
922,US-Twt,2017,3,1,3.576882,4.041596,5.130678,4.739996,9.119830,0.000,...,9.996,6.125646,8.174418,11.830005,9.004700,38.1087,Croplands,-7.0,Rice,Annual crops
923,US-Twt,2017,3,2,2.249392,2.251344,2.752283,3.215651,5.102038,0.900,...,11.239,4.881132,5.240039,6.314567,7.175264,38.1087,Croplands,-7.0,Rice,Annual crops


## calculate metrics index for monthly data each stations

In [23]:
result_for_monthly = result.groupby(["Site_ID", "Year", "Month"]).agg(
  number_row = ("ET_fill", "size"),
  number_value =  ("ET_fill", "count"),
)
result_for_monthly.reset_index(inplace=True)

result_for_monthly["percent_of_value"] = result_for_monthly["number_value"] / result_for_monthly["number_row"]


monthly_data = daily_data_1.groupby(['Site_ID', 'Year', 'Month']).agg({'aet_budyko_oudin': 'sum',
                                                                       'aet_budyko_hargreaves': 'sum',
                                                                       'aet_budyko_abtew': 'sum',
                                                                       'aet_budyko_mcguinness_bordne': 'sum',
                                                                       'ET_fill': 'sum',
                                                                       'ppt_ameri' : 'sum',
                                                                       'ASCE_ETo' : 'sum',
                                                                       'ppt' : 'sum',
                                                                       'oudin' : 'sum',
                                                                       'hargreaves' : 'sum',
                                                                       'abtew' : 'sum',
                                                                       'mcguinness_bordne' : 'sum'}).reset_index()


monthly_data = monthly_data.merge(
    stations_unique_info,
    left_on=['Site_ID'],
    right_on=['Site_ID'],
)


monthly_metrics = calculate_timescale_metrics(monthly_data, 'Monthly')


monthly_metrics = monthly_metrics.merge(
    stations_unique_info,
    left_on=['Site_ID'],
    right_on=['Site_ID'],
)



# monthly_metrics = pd.merge(monthly_metrics, result_for_monthly[['Site_ID', 'Year', 'Month', 'number_row', 'number_value', 'percent_of_value']],
#                   on=['Site_ID', 'Year', 'Month'],
#                   how='left')
monthly_data


,Site_ID,Year,Month,aet_budyko_oudin,aet_budyko_hargreaves,aet_budyko_abtew,aet_budyko_mcguinness_bordne,ET_fill,ppt_ameri,ASCE_ETo,ppt,oudin,hargreaves,abtew,mcguinness_bordne,latitude,General_classification,Elevation,Land_cover_details,Land_cover_type
0,Almond_High,2016,10,5.573189,6.394372,6.817488,7.369026,14.524432,37.592,6.034090,22.815,8.364811,9.833769,10.991679,12.296272,36.1697,Croplands,147.0,Almond,Orchards
1,Almond_High,2016,11,3.457981,4.215298,5.127005,4.580734,8.163006,34.544,5.988743,21.596,4.898145,6.435082,8.551687,7.200273,36.1697,Croplands,147.0,Almond,Orchards
2,Almond_High,2016,12,4.731302,5.249017,6.798754,6.190804,6.858637,39.116,4.393247,23.369,6.452863,8.013008,12.095018,9.485709,36.1697,Croplands,147.0,Almond,Orchards
3,Almond_High,2017,1,14.848838,17.150137,23.597011,20.940994,31.763826,182.372,15.648598,109.738,16.918288,19.394778,28.054068,24.869884,36.1697,Croplands,147.0,Almond,Orchards
4,Almond_High,2017,2,15.876717,16.352400,20.478806,21.716244,15.752455,75.692,11.246272,61.703,20.190877,21.947573,28.997933,29.680589,36.1697,Croplands,147.0,Almond,Orchards
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,US-Twt,2016,11,5.199281,5.333597,6.902376,7.107846,6.115304,29.400,6.254415,28.803,7.797869,8.524474,11.685886,11.462867,38.1087,Croplands,-7.0,Rice,Annual crops
430,US-Twt,2016,12,7.733652,8.818499,11.965617,10.497985,5.406630,85.600,2.398865,64.854,9.404073,11.425070,17.105943,13.823988,38.1087,Croplands,-7.0,Rice,Annual crops
431,US-Twt,2017,1,15.152333,15.427181,22.648911,21.743746,15.360403,229.900,10.091391,227.359,15.969643,16.515428,25.217126,23.475375,38.1087,Croplands,-7.0,Rice,Annual crops
432,US-Twt,2017,2,22.347142,20.645603,26.266093,31.124664,19.245805,87.600,9.992841,158.246,26.118873,25.501229,34.377244,38.394743,38.1087,Croplands,-7.0,Rice,Annual crops


## calculate metrics index for yearly data each stations

In [24]:
result_for_yearly = result.groupby(["Site_ID", "Year"]).agg(
  number_row = ("ET_fill", "size"),
  number_value =  ("ET_fill", "count"),
)
result_for_yearly.reset_index(inplace=True)




result_for_yearly["percent_of_value"] = result_for_yearly["number_value"] / result_for_yearly["number_row"]




yearly_data = daily_data_1.groupby(['Site_ID', 'Year']).agg({'aet_budyko_oudin': 'sum',
                                                                       'aet_budyko_hargreaves': 'sum',
                                                                       'aet_budyko_abtew': 'sum',
                                                                       'aet_budyko_mcguinness_bordne': 'sum',
                                                                       'ET_fill': 'sum',
                                                                       'ppt_ameri' : 'sum',
                                                                       'ASCE_ETo' : 'sum',
                                                                       'ppt' : 'sum',
                                                                       'oudin' : 'sum',
                                                                       'hargreaves' : 'sum',
                                                                       'abtew' : 'sum',
                                                                       'mcguinness_bordne' : 'sum'}).reset_index()


yearly_data = yearly_data.merge(
    stations_unique_info,
    left_on=['Site_ID'],
    right_on=['Site_ID'],
)


yearly_metrics = calculate_timescale_metrics(yearly_data, 'yearly')


yearly_metrics = yearly_metrics.merge(
    stations_unique_info,
    left_on=['Site_ID'],
    right_on=['Site_ID'],
)





# yearly_metrics = pd.merge(yearly_metrics, result_for_yearly[['Site_ID', 'Year','number_row', 'number_value', 'percent_of_value']],
#                   on=['Site_ID', 'Year'],
#                   how='left')
yearly_data


,Site_ID,Year,aet_budyko_oudin,aet_budyko_hargreaves,aet_budyko_abtew,aet_budyko_mcguinness_bordne,ET_fill,ppt_ameri,ASCE_ETo,ppt,oudin,hargreaves,abtew,mcguinness_bordne,latitude,General_classification,Elevation,Land_cover_details,Land_cover_type
0,Almond_High,2016,13.762473,15.858687,18.743246,18.140565,29.546076,111.252,16.416080,67.780,19.715819,24.281858,31.638384,28.982254,36.1697,Croplands,147.0,Almond,Orchards
1,Almond_High,2017,55.512167,60.423239,73.442554,71.540049,124.825823,341.884,86.570660,213.786,94.625267,114.280019,146.523608,139.099142,36.1697,Croplands,147.0,Almond,Orchards
2,Almond_High,2018,43.427222,48.435403,58.300243,56.850595,100.823938,373.380,59.858800,175.319,67.960010,84.730230,113.725019,99.901214,36.1697,Croplands,147.0,Almond,Orchards
3,Almond_High,2019,66.612334,76.541428,90.766073,85.059632,151.167258,197.612,74.275379,180.871,98.275354,122.669357,160.440760,144.464771,36.1697,Croplands,147.0,Almond,Orchards
4,Almond_Low,2016,17.837863,22.256750,27.509638,23.672260,26.623706,26.416,9.463387,108.381,24.851852,31.479157,41.505783,36.532222,36.9466,Croplands,78.0,Almond,Orchards
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,US-Twt,2013,34.918407,38.288048,45.969852,44.492503,58.597428,93.500,68.038036,130.365,69.371108,80.180563,103.429854,101.975529,38.1087,Croplands,-7.0,Rice,Annual crops
57,US-Twt,2014,68.401441,66.489577,82.317471,91.485500,82.399181,434.900,105.290600,419.235,105.833674,111.321533,144.179849,155.575501,38.1087,Croplands,-7.0,Rice,Annual crops
58,US-Twt,2015,41.849603,46.035721,57.271068,55.325717,65.788329,153.500,92.812371,178.294,73.871535,87.417927,111.049703,108.591157,38.1087,Croplands,-7.0,Rice,Annual crops
59,US-Twt,2016,81.760895,81.916927,101.347431,109.805154,87.470898,471.600,111.250847,497.814,128.655894,135.962989,178.211006,189.124165,38.1087,Croplands,-7.0,Rice,Annual crops


In [30]:

yearly_data.to_parquet("/content/drive/MyDrive/EC_Tower/result/yearly_data_central_valley.parquet", engine="pyarrow")
monthly_data.to_parquet("/content/drive/MyDrive/EC_Tower/result/monthly_data_central_valley.parquet", engine="pyarrow")
decade_data.to_parquet("/content/drive/MyDrive/EC_Tower/result/decade_data_central_valley.parquet", engine="pyarrow")
daily_data_1.to_parquet("/content/drive/MyDrive/EC_Tower/result/daily_data_central_valley.parquet", engine="pyarrow")

# organizing metrics index data frame

In [26]:
yearly_metrics[['rmse', 'nse', 'nrmse', 'r2']] = pd.DataFrame(yearly_metrics[0].tolist(), index=yearly_metrics.index)
monthly_metrics[['rmse', 'nse', 'nrmse', 'r2']] = pd.DataFrame(monthly_metrics[0].tolist(), index=monthly_metrics.index)
decade_metrics[['rmse', 'nse', 'nrmse', 'r2']] = pd.DataFrame(decade_metrics[0].tolist(), index=decade_metrics.index)
daily_metrics[['rmse', 'nse', 'nrmse', 'r2']] = pd.DataFrame(daily_metrics[0].tolist(), index=daily_metrics.index)


yearly_metrics = yearly_metrics.drop(0, axis=1)
monthly_metrics = monthly_metrics.drop(0, axis=1)
decade_metrics = decade_metrics.drop(0, axis=1)
daily_metrics = daily_metrics.drop(0, axis=1)

# clustering yearly metrics index data frame based on land cover

In [ ]:



yearly_data_based_Land_cover_type = yearly_metrics.groupby(['Site_ID', 'Year',"Method",'Land_cover_type']).agg({'rmse': 'mean',
                                                                                                                'nse': 'mean',
                                                                                                                'nrmse': 'mean',
                                                                                                                'r2': 'mean',
                                                                                                                'number_row' : 'mean',
                                                                                                                'number_value' : 'mean',
                                                                                                                'percent_of_value' : 'mean'}).reset_index()

yearly_data_based_Land_cover_type.describe()


,Year,rmse,nse,nrmse,r2,number_row,number_value,percent_of_value
count,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000
mean,2016.081967,26.148435,-0.026434,0.323288,0.902637,365.196721,315.131148,0.863010
std,2.548590,15.609261,1.063201,0.156973,0.112585,0.398337,83.013307,0.227501
min,2009.000000,4.807103,-3.637345,0.068693,0.599835,365.000000,82.000000,0.224044
25%,2014.000000,15.107696,-0.161708,0.233971,0.861952,365.000000,275.000000,0.753425
50%,2017.000000,21.784192,0.188819,0.297140,0.955772,365.000000,365.000000,1.000000
75%,2018.000000,38.803414,0.575971,0.395894,0.978251,365.000000,365.000000,1.000000
max,2020.000000,62.244324,0.965924,0.690605,1.000000,366.000000,366.000000,1.000000


# clustering monthly metrics index data frame based on land cover

In [ ]:
monthly_data_based_Land_cover_type = monthly_metrics.groupby(['Site_ID', 'Year','Month',"Method",'Land_cover_type']).agg({'rmse': 'mean',
                                                                                                                          'nse': 'mean',
                                                                                                                          'nrmse': 'mean',
                                                                                                                          'r2': 'mean',
                                                                                                                          'number_row' : 'mean',
                                                                                                                          'number_value' : 'mean',
                                                                                                                          'percent_of_value' : 'mean'}).reset_index()

monthly_data_based_Land_cover_type.describe()

,Year,Month,rmse,nse,nrmse,r2,number_row,number_value,percent_of_value
count,1736.000000,1736.000000,1736.000000,1736.000000,1736.000000,1736.000000,1736.000000,1736.000000,1736.000000
mean,2015.988479,6.336406,5.890319,0.288936,0.201094,0.656248,30.350230,30.198157,0.995065
std,2.531163,3.926379,2.369619,0.574238,0.075641,0.121481,0.896699,1.566919,0.043337
min,2009.000000,1.000000,3.154639,-1.638684,0.105078,0.403047,28.000000,10.000000,0.322581
25%,2014.000000,3.000000,4.405261,0.262775,0.152961,0.550237,30.000000,30.000000,1.000000
50%,2017.000000,5.000000,5.043414,0.443627,0.183443,0.641739,31.000000,31.000000,1.000000
75%,2018.000000,10.000000,6.903779,0.526773,0.221943,0.770300,31.000000,31.000000,1.000000
max,2020.000000,12.000000,12.213004,0.862404,0.423343,0.898498,31.000000,31.000000,1.000000


# clustering decade metrics index data frame based on land cover

In [ ]:
decade_data_based_Land_cover_type = decade_metrics.groupby(['Site_ID', 'Year','Month','decade_of_this_month',"Method",'Land_cover_type']).agg({'rmse': 'mean',
                                                                                                                                                 'nse': 'mean',
                                                                                                                                                 'nrmse': 'mean',
                                                                                                                                                 'r2': 'mean',
                                                                                                                                                 'number_row' : 'mean',
                                                                                                                                                 'number_value' : 'mean',
                                                                                                                                                 'percent_of_value' : 'mean'}).reset_index()

decade_data_based_Land_cover_type.describe()

,Year,Month,decade_of_this_month,rmse,nse,nrmse,r2,number_row,number_value,percent_of_value
count,3700.000000,3700.000000,3700.000000,3700.000000,3700.000000,3700.000000,3700.000000,3700.000000,3700.000000,3700.000000
mean,2016.050811,6.097297,1.998919,3.198579,0.133718,0.196638,0.502838,10.123243,10.114595,0.999165
std,2.537976,4.074362,0.811514,1.096759,0.592146,0.037270,0.130736,0.569525,0.591782,0.017218
min,2009.000000,1.000000,1.000000,1.952433,-1.939748,0.124870,0.183957,8.000000,5.000000,0.500000
25%,2014.000000,3.000000,1.000000,2.479862,0.117093,0.181526,0.429811,10.000000,10.000000,1.000000
50%,2017.000000,5.000000,2.000000,2.692819,0.297336,0.193571,0.503173,10.000000,10.000000,1.000000
75%,2018.000000,11.000000,3.000000,3.570550,0.430175,0.202944,0.593753,10.000000,10.000000,1.000000
max,2020.000000,12.000000,3.000000,6.319448,0.729712,0.300411,0.764869,11.000000,11.000000,1.000000
